In [ ]:
%load_ext autoreload
%autoreload 2

from data_prep import DataPreparation, filtering_data
from lstm import LSTMModelTrainerAttention
from rf import RandomForestTrainer
from lgbm import LightGBMTrainer
from catboost_trainer import CatBoostTrainer
import os
import dill as pickle
import numpy as np
import pandas as pd
from analysis import plot_correlation_heatmap, get_ETO_DEP
from lstm import LSTMModelTrainerAttention, calculate_and_plot_errors#, LSTMModelTrainer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker
from tqdm import tqdm
from lstm import LSTMRollingForecaster
from collections import defaultdict
import matplotlib.pyplot as plt
from helper import extract_flight_details
from lstm import LSTMModelTrainerAttention, LSTMRollingForecaster
from rf import RandomForestTrainer
from lgbm import LightGBMTrainer
from catboost_trainer import CatBoostTrainer
from transformer import TransformerModelTrainer
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
output= r'C:\Users\iLabs_6\Documents\Tex\realtimetest3'
modelname = 'newbaseline_atot2'

with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_df = pickle.load(f)

extended_df['cap_DEP'].fillna(extended_df['cap_DEP'].mean(), inplace=True)
extended_df['cap_DES'].fillna(extended_df['cap_DES'].mean(), inplace=True)
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_eobt', case=False)]
extended_df = extended_df.loc[:, ~extended_df.columns.str.contains('t_to_atot', case=False)]
extended_df = extended_df.drop_duplicates()

pd.set_option("display.max_columns", None)
X, y, colnames = filtering_data(extended_df, airport ='EHAM', save=True)
X = X.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)

modelname = 'newbaseline_etot2'


with  open(os.path.join(output, modelname), 'rb') as f:  
    extended_real= pickle.load(f)
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_eobt', case=False)]
extended_real = extended_real.loc[:, ~extended_real.columns.str.contains('t_to_atot', case=False)]
extended_real = extended_real.drop_duplicates()

pd.set_option("display.max_columns", None)
X_r, y_r, colnames = filtering_data(extended_real, airport ='EHAM', save=False)
X_r = X_r.drop(['ADES_EHAM', 'ADESLong', 'ADESLat', 'cbasentry'], axis =1)

modelpath= r"C:\Users\iLabs_6\Documents\Tex\AirTrafficDelays\LSTM_Models"

modelname = 'newbaseline-s6'

with  open(os.path.join(modelpath, modelname), 'rb') as f:  
    best_model = pickle.load(f)

In [ ]:
data_prep = DataPreparation()


X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, time_horizons, cbaslabels = data_prep.fit_transform_data(X, y)
X_real, ETOT_horizons, cbaslabels = data_prep.transform_data(X_r)

X_train, X_test, y_train, y_test = data_prep.fit_transform_data(X, y, split_ratio=0.8, mode='rf')


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

abs_ap = []
ap = []
lstm_se = []
lstm_y_true =[]
lstm_y_pred = []
target_length = 61  # 
start_idx, end_idx =int( 0.8*len(y_r)), -1  #all!

abs_error_dict = {}
error_dict = {}

e_adep = defaultdict(list)
e_dist =  defaultdict(list)
e_wd =  defaultdict(list)
e_time =  defaultdict(list)
adeplonglist = defaultdict(list)
adeplatlist = defaultdict(list)
adeplatlonlist = defaultdict(list)
actypelist = defaultdict(list)
flighttypelist = defaultdict(list)

eto_dict = {}
# Loop through the specified range in `y_r`
for fnr in tqdm(np.where((y_r <= 130))[0][start_idx:end_idx]):
    # Create rolling forecaster instance
    recursive = LSTMRollingForecaster(best_model, data_prep, X_real[fnr], ETOT_horizons)
    pred = recursive.rolling_forecast()
    # Calculate absolute error per timestep

    ### WR ATOT
    absolute_error_per_timestep = np.abs(pred[-target_length:] - y_r[fnr])
    error_per_timestep = (pred[-target_length:] - y_r[fnr])
    squared_error_per_timestep = (pred[-target_length:] - y_r[fnr]) ** 2

    # Ensure each error array has `target_length` elements by padding with NaNs if needed
    if len(absolute_error_per_timestep) < target_length:
        pad_length = target_length - len(absolute_error_per_timestep)

        padded_abs_error = np.pad(absolute_error_per_timestep, (0, target_length - len(absolute_error_per_timestep)), constant_values=np.nan)
        padded_error = np.pad(error_per_timestep, (0, target_length - len(error_per_timestep)), constant_values=np.nan)
        padded_sq_error = np.pad(squared_error_per_timestep, (0, pad_length), constant_values=np.nan)

        abs_ap.append(padded_abs_error)
        ap.append(padded_error)
        lstm_se.append(padded_sq_error)

    else:
        abs_ap.append(absolute_error_per_timestep)
        ap.append(error_per_timestep)
        lstm_se.append(squared_error_per_timestep)

    y_true = y_r[fnr] * np.ones(target_length)  # Repeat the scalar true value
    y_pred = pred[-target_length:]

    # Handle cases where there might not be enough predicted values
    if len(y_pred) < target_length:
        pad_length = target_length - len(y_pred)
        y_true_padded = y_true
        y_pred_padded = np.pad(y_pred, (0, pad_length), constant_values=np.nan)
    else:
        y_true_padded = y_true
        y_pred_padded = y_pred

    # Append the padded true and predicted values
    lstm_y_true.append(y_true_padded)
    lstm_y_pred.append(y_pred_padded)

    ### REST (CBAS)

    absolute_error_per_timestep = np.abs(pred[-target_length:] - y_r[fnr])[::-1]
    error_per_timestep = (pred[-target_length:] - y_r[fnr])[::-1]
    clabel = [x.astype('timedelta64[s]').astype(int) / 60 for x in cbaslabels[fnr] if not np.isnat(x)][-target_length:][::-1]

    if len(absolute_error_per_timestep) > len(clabel):
        absolute_error_per_timestep = absolute_error_per_timestep[-len(clabel):]
        error_per_timestep = error_per_timestep[-len(clabel):]


    inv  = data_prep.inverse_transform_single_flight(X_real[fnr])
    info = extract_flight_details(inv)
    etocols =inv[[c for c in inv.columns if 'etodep' in c]]

    eto_error = (etocols.iloc[0].tolist() - y_r[fnr])
    eto_e = eto_error[~np.isnan(eto_error)][::-1]
    
    # Loop over the filtered `clabel` and `absolute_error_per_timestep`
    for i in range(len(absolute_error_per_timestep)):
        time_to_cbas = np.round(clabel[i], 0)
        if time_to_cbas not in abs_error_dict:
            error_dict[time_to_cbas] = []
            abs_error_dict[time_to_cbas] = []
            eto_dict[time_to_cbas]=[]
        abs_error_dict[time_to_cbas].append(absolute_error_per_timestep[i])
        error_dict[time_to_cbas].append(error_per_timestep[i])
        try:

            eto_dict[time_to_cbas].append(eto_e[i])
        except:
            continue

    ### FOR ATOT GRAPTH ANA:LYYSY

    absolute_error_per_timestep = np.abs(pred[-target_length:] - y_r[fnr])
    error_per_timestep = pred[-target_length:] - y_r[fnr]

    # Ensure each error array has `target_length` elements by padding with NaNs if needed
    if len(absolute_error_per_timestep) < target_length:
        absolute_error_per_timestep = np.pad(absolute_error_per_timestep, (0, target_length - len(absolute_error_per_timestep)), constant_values=np.nan)

    e_adep[info['adep']].append(absolute_error_per_timestep[-61:])
    e_dist[info['distance']].append(absolute_error_per_timestep[-61:])
    e_wd[info['day']].append(absolute_error_per_timestep[-61:])
    e_time[info['ETOT']].append(absolute_error_per_timestep[-61:])
    adeplonglist[info['longitude']].append(absolute_error_per_timestep[-61:])
    adeplatlist[info['latitude']].append(absolute_error_per_timestep[-61:])
    adeplatlonlist[(info['latitude'], info['longitude'])].append(absolute_error_per_timestep)
    actypelist[info['actype']].append(absolute_error_per_timestep[-61:])
    flighttypelist[info['flighttype']].append(absolute_error_per_timestep[-61:])



lstm_se = np.array(lstm_se)           # Shape: (num_samples, target_length)
lstm_rmse_per_timestep = np.sqrt(np.nanmean(lstm_se, axis=0))
overall_rmse = np.sqrt(np.nanmean(lstm_se))


# Bucket the errors into 5-minute intervals
bucketed_abs_errors = defaultdict(list)
for time_in_minutes, errors in abs_error_dict.items():
    bucket = int(time_in_minutes // 5) * 5   # Group into 5-minute buckets
    bucketed_abs_errors[bucket].extend(errors)   # Aggregate errors within the bucket

bucketed_errors = defaultdict(list)
for time_in_minutes, errors in error_dict.items():
    bucket = int(time_in_minutes // 5) * 5   # Group into 5-minute buckets
    bucketed_errors[bucket].extend(errors)   # Aggregate errors within the bucket

eto_bucket_errors = defaultdict(list)
for time_in_minutes, errors in eto_dict.items():
    bucket = int(time_in_minutes // 5) * 5   # Group into 5-minute buckets
    eto_bucket_errors[bucket].extend(errors)   # Aggregate errors within the bucket

# Compute the mean and standard deviation of error for each bucket
mean_abs_errors_by_bucket = {bucket: np.nanmean(errors) for bucket, errors in bucketed_abs_errors.items()}
mean_errors_by_bucket = {bucket: np.nanmean(errors) for bucket, errors in bucketed_errors.items()}
std_errors_by_bucket = {bucket: np.nanstd(errors) for bucket, errors in bucketed_errors.items()}
eto_errors_by_bucket = {bucket: np.nanstd(errors) for bucket, errors in eto_bucket_errors.items()}

# Sort the buckets
sorted_abs_buckets = sorted(mean_abs_errors_by_bucket.items())
sorted_buckets = sorted(mean_errors_by_bucket.items())
sorted_std_buckets = sorted(std_errors_by_bucket.items())
sorted_eto_buckets = sorted(eto_errors_by_bucket.items())

# Extract sorted times and mean errors for plotting
abs_times = [item[0] for item in sorted_abs_buckets]
abs_errors = [item[1] for item in sorted_abs_buckets]
times = [item[0] for item in sorted_buckets]
errors = [item[1] for item in sorted_buckets]
std_errors = [item[1] for item in sorted_std_buckets]
eto_errors = [item[1] for item in sorted_eto_buckets]

# Plot Mean Absolute Prediction Error
plt.figure(figsize=(10, 6))
plt.plot(abs_times[:-10], abs_errors[:-10], marker='o', linestyle='-')
plt.title("Mean Absolute Prediction Error vs Time to CBAS Entry (bucketed in 5-minute intervals)")
plt.xlabel("Time to CBAS Entry (minutes)")
plt.ylabel("Mean Absolute Prediction Error (minutes)")
ax = plt.gca()
ax.invert_xaxis()

plt.grid(True)
plt.show()
ax = plt.gca()
ax.invert_xaxis()
time_values = [item[0] for item in sorted_buckets]
xt = [x for x in range(0,320,20)][::-1]
print(f'{xt=}')
plt.xticks(xt, fontsize=14)
plt.yticks(fontsize=14)

plt.figure(figsize=(10, 6))
plt.plot(times[:-10], eto_errors[:-10], marker='o', linestyle='-')
plt.title("Mean Absolute Prediction Error vs Time to CBAS Entry (bucketed in 5-minute intervals)")
plt.xlabel("Time to CBAS Entry (minutes)")
plt.ylabel("Mean Absolute Prediction Error (minutes)")
ax = plt.gca()
ax.invert_xaxis()

plt.grid(True)
plt.show()

# Plot Mean Prediction Error with Standard Deviation as Fill
plt.figure(figsize=(10, 6))
plt.plot(times[:-10], errors[:-10], marker='o', linestyle='-', label='Mean Error')
plt.fill_between(
    times[:-10],
    [e - s for e, s in zip(errors[:-10], std_errors[:-10])],
    [e + s for e, s in zip(errors[:-10], std_errors[:-10])],
    color='b',
    alpha=0.2,
    label='±1 Std Dev'
)


plt.title("Mean Prediction Error vs Time to CBAS Entry (bucketed in 5-minute intervals)")
plt.xlabel("Time to CBAS Entry (minutes)")
plt.ylabel("Mean Prediction Error (minutes)")
plt.legend()
ax = plt.gca()
ax.invert_xaxis()

plt.grid(True)
plt.show()

lstm_y_true = np.array(lstm_y_true)  # Shape: (num_samples, target_length)
lstm_y_pred = np.array(lstm_y_pred)  # Shape: (num_samples, target_length)

# Flatten arrays for overall R² calculation
flat_y_true = lstm_y_true.flatten()
flat_y_pred = lstm_y_pred.flatten()
valid_mask = ~np.isnan(flat_y_true) & ~np.isnan(flat_y_pred)

if np.any(valid_mask):
    overall_r2 = r2_score(flat_y_true[valid_mask], flat_y_pred[valid_mask])
else:
    overall_r2 = np.nan


overall_mae = np.nanmean(abs_ap[-61:])
overall_rmse = np.sqrt(np.nanmean(lstm_se[-61:]))

overall_std = np.nanstd(abs_ap[-61:])



lstm_r2_per_timestep = []
for t in range(target_length):
    y_true_t = lstm_y_true[:, t]
    y_pred_t = lstm_y_pred[:, t]
    mask_t = ~np.isnan(y_true_t) & ~np.isnan(y_pred_t)
    if np.sum(mask_t) > 1:
        r2_t = r2_score(y_true_t[mask_t], y_pred_t[mask_t])
    else:
        r2_t = np.nan
    lstm_r2_per_timestep.append(r2_t)
rf_r2_per_timestep = np.array(lstm_r2_per_timestep)
print(f'{rf_r2_per_timestep=}')
plt.figure(figsize=(12, 6))
plt.plot(ETOT_horizons[:target_length], lstm_r2_per_timestep, marker='s', color='green', label='R²')
plt.xlabel('Timestep')
plt.xticks(ETOT_horizons[:target_length], rotation='vertical')
plt.ylabel('R²')
plt.title('Random Forest: R² for Each Timestep')
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()


print("Raw True Values (y_true):", lstm_y_true)
print("Raw Predictions (y_pred):", lstm_y_pred)
print(f"Overall MAE: {overall_mae:.4f}")
print(f"Overall RMSE: {overall_rmse:.4f}")
print(f"Overall R²: {overall_r2:.4f}")
print(f"Overall Std Deviation of MAE: {overall_std:.4f}")
  
# Calculate mean absolute error (MAE) per timestep, ignoring NaNs in the calculation

print(f'{lstm_rmse_per_timestep=}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from collections import defaultdict

warnings.filterwarnings("ignore", category=UserWarning)

day_mapping = {
    'day_of_week_0': 'Monday',
    'day_of_week_1': 'Tuesday',
    'day_of_week_2': 'Wednesday',
    'day_of_week_3': 'Thursday',
    'day_of_week_4': 'Friday',
    'day_of_week_5': 'Saturday',
    'day_of_week_6': 'Sunday'
}

# Function to pad or truncate error lists for consistency
def pad_errors(errors, target_length=61):
    padded_errors = []
    for error in errors:
        current_length = len(error)
        if current_length < target_length:
            nans_to_append = target_length - current_length
            padded_error = np.concatenate([error, [np.nan]*nans_to_append])
        elif current_length > target_length:
            padded_error = error[:target_length]
        else:
            padded_error = error
        padded_errors.append(padded_error)
    return np.array(padded_errors)

# Function to plot a dictionary of errors with day names
def plot_dict_errors(error_dict, title, target_length=61):
    plt.figure(figsize=(10, 6))
    maelist = []
    
    # Sort the keys based on day number to ensure chronological order
    sorted_keys = sorted(
        error_dict.keys(),
        key=lambda x: int(x.split('_')[-1])  # Extract the day number for sorting
    )
    
    for key in sorted_keys:
        errors = error_dict[key]
        unique_lengths = set([len(x) for x in errors])
        print(f'Key: {key}, Lengths: {unique_lengths}')
        
        # Pad errors to target_length
        padded_errors = pad_errors(errors, target_length)
        
        try:
            mae_per_timestep = np.nanmean(padded_errors, axis=0)
            maelist.append(mae_per_timestep)
            day_name = day_mapping.get(key, key)  # Replace key with day name
            print(day_mapping[key], mae_per_timestep[0], mae_per_timestep[24], mae_per_timestep[48])
            plt.plot([-int(x) for x in ETOT_horizons[:target_length]], mae_per_timestep, marker='o', label=day_name)
        except ValueError as ve:
            print(f"Error processing key '{key}': {ve}")
            continue

    if maelist:
        mean_mae = np.nanmean(maelist, axis=0)
        print(f'{mean_mae=}')
        plt.plot([-int(x) for x in ETOT_horizons[:target_length]], mean_mae, marker='', label='Mean', color='red')

    plt.xlabel('Timestep')
    xt = [x for x in range(0,320,20)][::-1]
    yt = [x for x in range(0,16,2)]

    plt.xticks(xt, fontsize=14)
    plt.yticks(yt, fontsize=14)
    plt.ylabel('Mean Absolute Error (MAE)')
    ax = plt.gca()
    ax.invert_xaxis()
    plt.legend(title="Days of the Week")
    plt.tight_layout()
    plt.grid(True)
    plt.show()

print(f'{e_wd.keys()=}')
plot_dict_errors(e_wd, "MAE per Timestep - Day")

bucketed_errors = defaultdict(list)

for time_key, eees in e_time.items():
    time_to_cbas = time_key  # Use time_key as time to CBAS (assuming time_key is in minutes)
    bucket = int(time_to_cbas//180 )  *3 # Group into 60 minute intervals
    bucketed_errors[bucket].extend(eees)


bucket_mapping = {
    0: "0:00-3:00",
    3: "3:00-6:00",
    6: "6:00-9:00",
    9: "9:00-12:00",
    12: "12:00-15:00",
    15: "15:00-18:00",
    18: "18:00-21:00",
    21: "21:00-0:00"
}
plt.figure(figsize=(10, 6))

print(f'{sorted(bucketed_errors.keys())=}')
for bucket, edict in sorted(bucketed_errors.items()):
    print(f'{bucket=}')
    if bucket in [0, 21]:
        continue
    print(f'{len(edict)=}')
    # print(f'{edic]=}')
    edict=pad_errors(edict)
    mae_per_timestep = np.nanmean(edict, axis=0)

    plt.plot([-int(x) for x in ETOT_horizons[:target_length]], mae_per_timestep, marker='o', label=bucket_mapping[bucket])
plt.xlabel('Timestep')
xt = [x for x in range(0,320,20)][::-1]
yt = [x for x in range(0,16,2)]

plt.xticks(xt, fontsize=14)
plt.yticks(yt, fontsize=14)
plt.ylabel('Mean Absolute Error (MAE)')
ax = plt.gca()
ax.invert_xaxis()
plt.legend(
    loc="upper center",
    bbox_to_anchor=(0.45, -0.15),
    ncol=3,  # One column for clarity
    frameon=False,
    fontsize=14
)
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Create a DataFrame for the plot
print(f'{actypelist}')
data = pd.DataFrame({'Aircraftype': [x.split('actype_')[1] for x in actypelist.keys()], 'MAE': [np.nanmean(x) for x in actypelist.values()], 'flightcount': [len(x) for x in actypelist.values()]})

print(f'{data=}')
# Number of airports
N = len(data)

# Compute angle for each bar
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Complete the loop

# Heights of bars
heights = data['MAE'].tolist()
heights += heights[:1]  # Complete the loop

# Initialize the plot
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)

# Remove labels and ticks
ax.set_xticks([])
ax.set_yticks([])

# Set the minimum radius (distance from center where bars start)
bottom = 2.0  # Adjust this value to change how far bars start from the center

# Add bars
bar_width = 2 * np.pi / N * 0.8  # Adjust the 0.8 factor to change bar width (spacing between bars)
bars = ax.bar(angles[:-1], heights[:-1], width=bar_width, bottom=bottom, color='salmon', alpha=0.6)

# Adjust the maximum radius to ensure all bars and labels fit
max_height = bottom + max(heights) + max(heights) * 0.1  # Add some padding
ax.set_ylim(0, max_height)

# Add labels
for i, bar in enumerate(bars):
    angle = angles[i]
    rotation = np.degrees(angle)
    alignment = ''
    if rotation >= 270 or rotation <= 90:
        alignment = 'left'
        rotation = rotation
    else:
        alignment = 'right'
        rotation = rotation + 180
    # Position the label at the top of the bar
    ax.text(
        angle,
        bottom + heights[i] + max(heights) * 0.05,  # Adjust position based on 'bottom' and 'heights'
        data['Aircraftype'].iloc[i],
        ha=alignment,
        va='center',
        rotation=rotation,
        rotation_mode='anchor',
        fontsize=8
    )

plt.title('Circular Barplot of MAE per Airport', y=1.08)
plt.show()
pd.set_option('display.max_rows', None)  # Show all rows
print(data)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define a threshold for flight count to group small aircraft
flight_count_threshold = 50

# Group small aircraft under 'Other'
data_grouped = data.copy()
data_grouped.loc[data_grouped['flightcount'] < flight_count_threshold, 'Aircraftype'] = 'Other'

# Aggregate the grouped data by summing flight counts and calculating the average MAE
grouped_data = data_grouped.groupby('Aircraftype', as_index=False).agg({
    'flightcount': 'sum',  # Sum up flight counts for 'Other'
    'MAE': 'mean'          # Use average MAE for 'Other'
})

# Sort the data for better visualization
grouped_data = grouped_data.sort_values(by='flightcount', ascending=False)

fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar chart for flight count
bar1 = ax1.bar(grouped_data['Aircraftype'], grouped_data['flightcount'], 
               color='gray', alpha=0.6, label='Flight Count')
ax1.set_xlabel('Aircraft Type')
ax1.set_ylabel('Flight Count', color='gray')
ax1.tick_params(axis='y', labelcolor='gray')

# Align grid on the primary y-axis
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

# Line plot for MAE
ax2 = ax1.twinx()
ax2.plot(grouped_data['Aircraftype'], grouped_data['MAE'], 
         color='red', marker='o', label='Mean Absolute Error (MAE)')
ax2.set_ylabel('Mean Absolute Error (MAE)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
yt = [x for x in range(0,20,2)]
ax2.set_yticks(yt)
ax2.grid(False)

# Add legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

plt.xticks(rotation=90)
fig.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm


# Sort e_adep by the length of the flight list
sorted_e_adep = dict(sorted(e_adep.items(), key=lambda item: len(item[1]), reverse=True))

# Take the top 25 airports
top_e_adep = {k: v for k, v in list(sorted_e_adep.items())[:25]}

# Flatten the data for boxplot
data = []
for airport, flight_errors in top_e_adep.items():
    for flight_error in flight_errors:
        for timestep_error in flight_error:
            data.append({"Airport": airport.split('_')[1], "MAE": timestep_error})

df = pd.DataFrame(data)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# Customize outlier marker properties
flierprops = {
    "marker": "o",
    "markersize": 0.1,  # Slightly larger markers for better visibility
    "markerfacecolor": "red",  # Highlight outliers in red
    "markeredgecolor": "black",  # Black outline for contrast
    "linewidth": 0.5,
}

# Create boxplot
plt.figure(figsize=(14, 8))  # Larger figure size for clarity
sns.boxplot(
    data=df,
    x="Airport",
    y="MAE",
    palette="coolwarm",  # Use a colormap for visual differentiation
    linewidth=1.5,  # Make lines slightly thicker for better visibility
    showfliers=True,  # Show outliers
    flierprops=flierprops,  # Apply custom outlier properties
    medianprops={"color": "green", "linewidth": 2},  # Make the median line stand out
    boxprops={"facecolor": "none", "edgecolor": "black", "linewidth": 1.5},  # Transparent boxes with bold edges
    whiskerprops={"color": "black", "linewidth": 1.5},  # Bold whiskers
    capprops={"color": "black", "linewidth": 1.5},  # Bold caps
)

# Add a horizontal line at 0 for reference
plt.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.7)

# Add labels and title
plt.xlabel("Airports", fontsize=14, labelpad=10)
plt.ylabel("Error", fontsize=14, labelpad=10)
plt.ylim(-100,100)
# Customize tick labels
plt.xticks(rotation=90, fontsize=12, ha="right")  # Angled ticks with right alignment for readability
plt.yticks(fontsize=12)

# Adjust layout for better visualization
plt.tight_layout()

# Save high-quality plot
plt.savefig("mae_per_airport_outliers_boxplot.svg", format="svg", dpi=300, bbox_inches="tight")

# Show the plot
plt.show()


In [ ]:
max_airports = 50
data_grid = data.iloc[:max_airports].reset_index(drop=True)
num_airports = len(data_grid)

# Compute circle sizes (adjust scaling factor as needed)
max_mae = data_grid['MAE'].max()
sizes = data_grid['MAE'] / max_mae * 3000  # Adjust the multiplier to change circle sizes

# Normalize MAE values for color mapping
norm = mcolors.Normalize(vmin=data_grid['MAE'].min(), vmax=data_grid['MAE'].max())
cmap = cm.get_cmap('coolwarm')
colors = [cmap(norm(value)) for value in data_grid['MAE']]

# Grid dimensions
grid_rows = 5
grid_cols = 10

fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(grid_cols*1.5, grid_rows*1.5))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < num_airports:
        mae_value = data_grid['MAE'].iloc[i]
        color = colors[i]
        ax.scatter(0.5, 0.5, s=sizes.iloc[i], color=color, alpha=0.6)
        ax.text(0.5, 0.5, f"{mae_value:.1f}", ha='center', va='center', fontsize=12)
        ax.axis('off')  # Remove the box around the circle
        ax.set_title(data_grid['Airport'].iloc[i], fontsize=10)
    else:
        ax.axis('off')  # Hide unused subplots

plt.suptitle('MAE per Airport', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:

bnames =data_prep.basenames
# Plot feature importance
input_size = data_prep.input_size

model_trainer = LSTMModelTrainerAttention(data_prep=data_prep, input_size=input_size, model_type='varattention')

baseline = model_trainer.compute_baseline_mae(best_model, X_test_tensor[:10000], y_test_tensor[:10000], data_prep.scaler_y, time_horizons)
imp_df = model_trainer.compute_permutation_importance(best_model, X_test_tensor[:10000], y_test_tensor[:10000], data_prep.scaler_y, time_horizons, bnames)

from sklearn.preprocessing import MinMaxScaler
def normalize_dataframe_minmax(df, feature_range=(0, 1)):
    """
    Normalize the entire DataFrame using MinMaxScaler.
    
    :param df: pandas DataFrame to normalize
    :param feature_range: tuple (min, max) desired range of transformed data
    :return: Normalized DataFrame, fitted scaler
    """
    scaler = MinMaxScaler(feature_range=feature_range)
    scaled_array = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_array, columns=df.columns, index=df.index)
    return scaled_df, scaler

impdf2, scaler = normalize_dataframe_minmax(imp_df)
model_trainer.plot_permutation_importance(impdf2)
# model_trainer.plot_permutation_importance_with_highlights_interactive(imp_df, ['ko', 'visibility','etodepdelay', 'TOBTdelay', 'TSATdelay'])

In [ ]:
model_trainer.plot_permutation_importance(imp_df)


In [ ]:
def plot_top_features_line(importance_df, top_n=6, relative=False):
    """
    Plot the permutation feature importance as a line plot for the top N features.

    :param importance_df: DataFrame containing feature importance scores
    :param top_n: Number of top features to display in the plot
    :param relative: If True, plot relative importance percentages instead of absolute values
    """
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Set Seaborn style for professional-looking plots
    sns.set_theme(style="whitegrid")

    # Filter numeric columns only
    numeric_columns = importance_df.columns[pd.to_numeric(importance_df.columns, errors='coerce').notnull()]
    importance_df = importance_df[numeric_columns]

    # Calculate total importance and select top N features
    importance_df['total_importance'] = importance_df.sum(axis=1)
    top_features = importance_df.sort_values('total_importance', ascending=False).head(top_n)
    importance_df_top = top_features.drop('total_importance', axis=1)
    
    # Rename feature names for readability
    importance_df_top = importance_df_top.rename(index={
        'etodepdelay': 'Flight Plan Delay',
        'ko': 'Knock-on Delay',
        'modeltyp_ACT': 'Modeltyp ACT',
        'fltstate_SI': 'Flightstate SI',
        'TSATdelay': 'TSAT Delay'
    })
    
    # Compute relative importance percentages if requested
    if relative:
        # Compute total importance per timestep
        total_importance_per_timestep = importance_df_top.sum(axis=0)

        # Avoid division by zero
        total_importance_per_timestep[total_importance_per_timestep == 0] = 1e-8

        # Normalize each feature's importance by the total importance at each timestep
        importance_df_top = importance_df_top.divide(total_importance_per_timestep, axis=1) * 100

    # Create the figure
    plt.figure(figsize=(10, 6))
    markers = ['o', 's', 'D', '^', 'v', 'P', 'X', '*']

    # Plot using Seaborn for better aesthetics
    for i,feature in enumerate(importance_df_top.index):
        sns.lineplot(
            x=importance_df_top.columns.astype(float),
            y=importance_df_top.loc[feature],
            label=feature,
            linewidth=1,  # Thicker lines for academic plots
            markersize=8,
            # marker = markers[i]
        )

    # Add labels and grid
    ylabel = 'Relative Importance (%)' if relative else 'Feature Importance'
    plt.xlabel('Time to ATOT (minutes)', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    # Place the legend below the plot in two rows
    plt.legend(
        loc="upper center",
        bbox_to_anchor=(0.45, -0.1),  # Center horizontally and adjust vertical spacing
        ncol=3,  # 3 items in the first row
        frameon=False,
        fontsize=16
    )

    # Adjust layout and save
    plt.tight_layout()
    plt.grid(alpha=0.5)

    # Save the figure with high resolution
    plt.savefig("top_features_lineplot.png", format="png", dpi=300, bbox_inches="tight")

    # Show the plot
    plt.show()


def plot_top_features_area(importance_df, top_n=5):
    """
    Plot the permutation feature importance as a stacked area plot with relative importance percentages
    for the top N features.

    :param importance_df: DataFrame containing feature importance scores
    :param top_n: Number of top features to display in the plot
    """
    import matplotlib.pyplot as plt
    import numpy as np

    # Filter numeric columns only
    numeric_columns = importance_df.columns[pd.to_numeric(importance_df.columns, errors='coerce').notnull()]
    importance_df = importance_df[numeric_columns]

    # Calculate total importance and select top N features
    importance_df['total_importance'] = importance_df.sum(axis=1)
    top_features = importance_df.sort_values('total_importance', ascending=False).head(top_n).index
    importance_df_top = importance_df.loc[top_features].drop('total_importance', axis=1)

    # Compute total importance per timestep
    total_importance_per_timestep = importance_df_top.sum(axis=0)

    # Avoid division by zero
    total_importance_per_timestep[total_importance_per_timestep == 0] = 1e-8

    # Compute relative importance percentages per timestep
    importance_df_relative = importance_df_top.divide(total_importance_per_timestep, axis=1) * 100

    # Prepare data for stacking
    timesteps = importance_df_relative.columns.astype(float)
    features = importance_df_relative.index
    importance_values = importance_df_relative.values

    # Sort timesteps in case they're not in order
    sorted_indices = np.argsort(timesteps)
    timesteps = timesteps[sorted_indices]
    importance_values = importance_values[:, sorted_indices]

    # Plot stacked area chart
    plt.figure(figsize=(12, 8))
    plt.stackplot(timesteps, importance_values, labels=features)

    plt.xlabel('Timestep')
    plt.ylabel('Relative Importance (%)')
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    plt.tight_layout()
    plt.show()

def plot_top_features_heatmap(importance_df, top_n=5, relative=False):
    """
    Plot the permutation feature importance as a heatmap for the top N features.

    :param importance_df: DataFrame containing feature importance scores
    :param top_n: Number of top features to display in the plot
    :param relative: If True, plot relative importance percentages instead of absolute values
    """
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Set Seaborn style for professional-looking plots
    sns.set_theme(style="whitegrid")

    # Filter numeric columns only
    numeric_columns = importance_df.columns[pd.to_numeric(importance_df.columns, errors='coerce').notnull()]
    importance_df = importance_df[numeric_columns]

    # Calculate total importance and select top N features
    importance_df['total_importance'] = importance_df.sum(axis=1)
    top_features = importance_df.sort_values('total_importance', ascending=False).head(top_n)
    importance_df_top = top_features.drop('total_importance', axis=1)

    # Rename feature names for readability
    importance_df_top = importance_df_top.rename(index={
        'etodepdelay': 'Flight Plan Delay',
        'ko': 'Knock-on Delay',
        'modeltyp_ACT': 'Modeltyp ACT',
        'fltstate_SI': 'Flightstate SI',
        'TSATdelay': 'TSAT Delay'
    })

    # Compute relative importance percentages if requested
    if relative:
        # Compute total importance per timestep
        total_importance_per_timestep = importance_df_top.sum(axis=0)

        # Avoid division by zero
        total_importance_per_timestep[total_importance_per_timestep == 0] = 1e-8

        # Normalize each feature's importance by the total importance at each timestep
        importance_df_top = importance_df_top.divide(total_importance_per_timestep, axis=1) * 100

    # Create the heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(
        importance_df_top,
        cmap="coolwarm",
        linewidths=0.5,
        linecolor="gray",
        cbar_kws={'label': 'Relative Importance (%)' if relative else 'Feature Importance'},
        xticklabels=True,
        yticklabels=True
    )

    # Add labels
    plt.xlabel('Time to ATOT (minutes)', fontsize=16)
    plt.ylabel('Features', fontsize=16)
    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)

    # Title and layout
    plt.title('Feature Importance Over Time', fontsize=18, weight='bold')
    plt.tight_layout()

    # Save the figure
    plt.savefig("feature_importance_heatmap.png", format="png", dpi=300, bbox_inches="tight")

    # Show the plot
    plt.show()

numeric_columns = imp_df.columns[pd.to_numeric(imp_df.columns, errors='coerce').notnull()]
importance_df = imp_df[numeric_columns]

plot_top_features_line(importance_df, relative=True)
plot_top_features_area(importance_df)
plot_top_features_heatmap(importance_df, top_n=7, relative=True)